In [1]:
from transformers import AutoTokenizer, BertForMultipleChoice
import pandas as pd
#from transformers import pipeline
from tqdm import tqdm
from datasets import load_dataset

# restrict trainer to not use all cores
import os
os.environ["OPENMP_NUM_THREADS"] = "1"

2025-11-10 14:22:53.263124: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-10 14:22:53.359032: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-10 14:22:55.515475: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
os.environ["OPENMP_NUM_THREADS"]

'1'

In [3]:
dataset = load_dataset("../data/clean")
                       #names=["Tweet Id","Entity","Sentiment","Tweet Content"]
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'Tweet Id', 'Entity', 'Sentiment', 'Tweet Content'],
        num_rows: 160028
    })
    validation: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'Tweet Id', 'Entity', 'Sentiment', 'Tweet Content'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['Unnamed: 0.1', 'Unnamed: 0', 'Tweet Id', 'Entity', 'Sentiment', 'Tweet Content'],
        num_rows: 1000
    })
})

In [4]:
dataset = dataset.remove_columns(['Unnamed: 0.1', 'Unnamed: 0', 'Tweet Id'])
#dataset = dataset.

sentiments = ["Irrelevant","Positive","Neutral","Negative"]

print("Encoding sentiment")

for subset in ["train","validation","test"]:
    labels = []
    for d in tqdm(dataset[subset]["Sentiment"], desc=subset):
        for s in range(len(sentiments)):
            if sentiments[s] == d:
                labels.append(s)
                break
    dataset[subset] = dataset[subset].add_column("labels",labels)

print("Combining entities and text")
for subset in ["train","validation","test"]:
    combined = []
    for d in tqdm(range(len(dataset[subset])), desc=subset):
        combined.append(dataset[subset]["Entity"][d] + ". " + dataset[subset]["Tweet Content"][d])
    dataset[subset] = dataset[subset].add_column("inputs",combined)

dataset = dataset.remove_columns(["Entity","Tweet Content"])

dataset

Encoding sentiment


test: 100%|██████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 29672.13it/s]


Combining entities and text


test: 100%|████████████████████████████████████████████████████████████████████| 1000/1000 [00:01<00:00, 825.16it/s]


DatasetDict({
    train: Dataset({
        features: ['Sentiment', 'labels', 'inputs'],
        num_rows: 160028
    })
    validation: Dataset({
        features: ['Sentiment', 'labels', 'inputs'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['Sentiment', 'labels', 'inputs'],
        num_rows: 1000
    })
})

In [10]:
#dataset = dataset.remove_columns(["Sentiment"])
dataset.save_to_disk("bert-ds")
dataset

Saving the dataset (0/1 shards):   0%|          | 0/160028 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1500 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Sentiment', 'labels', 'inputs'],
        num_rows: 160028
    })
    validation: Dataset({
        features: ['Sentiment', 'labels', 'inputs'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['Sentiment', 'labels', 'inputs'],
        num_rows: 1000
    })
})

In [11]:
dataset["train"][0:4]

{'Sentiment': ['Negative', 'Neutral', 'Negative', 'Neutral'],
 'labels': [3, 2, 3, 2],
 'inputs': ['CallOfDuty. ... This is chilling',
  'Google. one',
  'ApexLegends. literally toxic bro, came out of the game when I was clear to him. Oh and I hit the same person three times to talk about lmao disappointment.',
  'WorldOfCraft. Take a look at this article I just got! [Uncanny Combat Gloves of the Incomparable Fighter]']}

In [12]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [15]:
# adapted from https://huggingface.co/docs/transformers/tasks/multiple_choice

count = 0
total = int(len(dataset["train"])/1000)

def preprocess(examples):
    global count, total
    count += 1
    tweet_text = [[content] * 4 for content in examples["inputs"]]
    #entities = examples["Entity"]
    
    tweet_sentiment = [
        [*sentiments] for label in examples["labels"]
    ]

    tweet_text = sum(tweet_text, [])
    tweet_sentiment = sum(tweet_sentiment, [])

    print(count,"/",total,"\t", len(tweet_text), len(tweet_sentiment))

    tokenized_examples = tokenizer(tweet_text, tweet_sentiment, truncation = True)
    return {k: [v[i : i+4] for i in range(0,len(v),4)] for k, v in tokenized_examples.items()}

In [16]:
#pbar = tqdm(total=len(dataset))
tokenized_data = dataset.map(preprocess, batched = True)

Map:   0%|          | 0/160028 [00:00<?, ? examples/s]

1 / 160 	 4000 4000
2 / 160 	 4000 4000
3 / 160 	 4000 4000
4 / 160 	 4000 4000
5 / 160 	 4000 4000
6 / 160 	 4000 4000
7 / 160 	 4000 4000
8 / 160 	 4000 4000
9 / 160 	 4000 4000
10 / 160 	 4000 4000
11 / 160 	 4000 4000
12 / 160 	 4000 4000
13 / 160 	 4000 4000
14 / 160 	 4000 4000
15 / 160 	 4000 4000
16 / 160 	 4000 4000
17 / 160 	 4000 4000
18 / 160 	 4000 4000
19 / 160 	 4000 4000
20 / 160 	 4000 4000
21 / 160 	 4000 4000
22 / 160 	 4000 4000
23 / 160 	 4000 4000
24 / 160 	 4000 4000
25 / 160 	 4000 4000
26 / 160 	 4000 4000
27 / 160 	 4000 4000
28 / 160 	 4000 4000
29 / 160 	 4000 4000
30 / 160 	 4000 4000
31 / 160 	 4000 4000
32 / 160 	 4000 4000
33 / 160 	 4000 4000
34 / 160 	 4000 4000
35 / 160 	 4000 4000
36 / 160 	 4000 4000
37 / 160 	 4000 4000
38 / 160 	 4000 4000
39 / 160 	 4000 4000
40 / 160 	 4000 4000
41 / 160 	 4000 4000
42 / 160 	 4000 4000
43 / 160 	 4000 4000
44 / 160 	 4000 4000
45 / 160 	 4000 4000
46 / 160 	 4000 4000
47 / 160 	 4000 4000
48 / 160 	 4000 4000
4

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

162 / 160 	 4000 4000
163 / 160 	 2000 2000


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

164 / 160 	 4000 4000


In [18]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['Sentiment', 'labels', 'inputs', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160028
    })
    validation: Dataset({
        features: ['Sentiment', 'labels', 'inputs', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['Sentiment', 'labels', 'inputs', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [19]:
from transformers import DataCollatorForMultipleChoice
collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)

In [20]:
import evaluate

accuracy = evaluate.load("accuracy")

In [21]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

model = AutoModelForMultipleChoice.from_pretrained("google-bert/bert-base-uncased")

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Training

In [ ]:
training_args = TrainingArguments(
    output_dir="../models",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["validation"],
    processing_class=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
model.save_pretrained("../models")

In [ ]:
a = tokenized_data["train"]

In [ ]:
a.rename_column("Unnamed: 0",'labels')

In [ ]:
tokenized_data = tokenized_data.rename_column("Unnamed: 0",'labels')

In [23]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['Sentiment', 'labels', 'inputs', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160028
    })
    validation: Dataset({
        features: ['Sentiment', 'labels', 'inputs', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1500
    })
    test: Dataset({
        features: ['Sentiment', 'labels', 'inputs', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
dataset = dataset.rename_column("label","labels")